# TREE OF THOUGHTS IMPLEMENTATION

***Remember to enter your project path and OpenAI API key before executing the code***

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/', force_remount=True)

os.chdir(PROJECT_PATH)

Mounted at /content/drive/


In [ ]:
PROJECT_PATH = <YOUR-PROJECT-PATH>
os.environ['OPENAI_API_KEY'] = <YOUR-OPENAI-API-KEY>

In [2]:
!ls

requirements.txt  tree-of-thoughts-using-langchain.ipynb


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.28.1
    Uninstalling openai-1.28.1:
      Successfully uninstalled openai-1.28.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.305
    Uninstalling langchain-0.0.305:
      Successfully uninstalled langchain-0.0.305


In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

In [5]:
# Step 1 - Building LLM to draft solutions
solutions_template = """
Step 1:
I'm facing a challenge related to {topic}. Can you generate two plans for implementing {topic}?
"""

solutions_prompt = PromptTemplate(
    input_variables = ["topic"],
    template = solutions_template
)
solutions_llm  = ChatOpenAI(openai_api_key = os.getenv('OPENAI_API_KEY'), temperature = 0.1)
solutions_chain = solutions_prompt | solutions_llm



# Step 2 - Build LLM to analyse each solution drafted in the previous step
analysis_template  = """
Step 2:
For each of the {solutions}, consider the advantages and disadvantages, ways to integrate them with existing urban infrastructure and planning strategies.
Your output should be in the form of description of the plan, advantages, disadvantages and planning strategies in a JSON format for each solution.
"""

analysis_prompt = PromptTemplate(input_variables = ["solutions"],
                                 template = analysis_template)
analysis_llm = ChatOpenAI(openai_api_key = os.getenv('OPENAI_API_KEY'), temperature = 0.1)
analysis_chain = analysis_prompt | analysis_llm


# Step 3 -  Build LLM to evaluate and rank solutions
evaluation_template = """
Step 3:

Evaluate and rank solutions based on {solutions} and {analysis} in previous steps,
rank the plans on a scale of 10 based on safety, environmental impact, reliability and cost-effectiveness and justify the rank.
Your output should be in the form of description about the plan, advantages, disadvantages, planning strategies,
evaluation with reason in a JSON format for each solution.
"""
evaluation_prompt = PromptTemplate(input_variables = ["solutions", "analysis"],
                                   template = evaluation_template)
evaluation_llm = ChatOpenAI(openai_api_key = os.getenv('OPENAI_API_KEY'), temperature = 0.1)
evaluation_chain = evaluation_prompt | evaluation_llm

# Build a chain of LLM

In [6]:
complete_chain = RunnablePassthrough.assign(solutions = solutions_chain)| RunnablePassthrough.assign(analysis = analysis_chain) | evaluation_chain

In [7]:
response = complete_chain.invoke({"topic": "air taxi"})
print(response.content)

{
  "Plan 1": {
    "description": "Plan 1 focuses on research and development, partnership and collaboration, marketing and promotion, operational setup, and expansion and growth of air taxi services.",
    "advantages": [
      "Thorough research ensures safety and efficiency of air taxi vehicles",
      "Partnerships leverage expertise and resources from various industries",
      "Strong marketing strategy raises awareness and attracts customers",
      "Established operational hubs and safety protocols ensure smooth operations",
      "Continuous improvement and expansion cater to growing market demand"
    ],
    "disadvantages": [
      "High initial investment required for research, development, and operational setup",
      "Dependence on partnerships may lead to conflicts or delays",
      "Marketing costs can be high and may not guarantee immediate returns",
      "Expansion may be limited by regulatory approvals and infrastructure constraints",
      "Competition from other